In [ ]:
import ollama
import torch
import re
from sentence_transformers import SentenceTransformer, util

def check_cuda():
    if torch.cuda.is_available():
        print("CUDA is available. GPU in use:", torch.cuda.get_device_name(0))
    else:
        print("CUDA is not available. Running on CPU.")

def get_similar_sentence(paragraph, query):
    """
    Find the most similar sentence in the paragraph to the user's query.
    """
    # Initialize SentenceTransformer model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Split paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph.strip())

    # Encode all sentences in the paragraph
    paragraph_embeddings = model.encode(sentences, convert_to_tensor=True)

    # Encode the user's query
    query_embedding = model.encode(query, convert_to_tensor=True)

    # Calculate cosine similarity between the query and each sentence in the paragraph
    cosine_scores = util.pytorch_cos_sim(query_embedding, paragraph_embeddings)[0]

    # Get the sentence with the highest cosine similarity
    best_match_index = torch.argmax(cosine_scores)
    best_match_sentence = sentences[best_match_index]

    return best_match_sentence

def ask_question(paragraph, question):
    # Find the most relevant sentence from the paragraph for the user's question
    context = get_similar_sentence(paragraph, question)

    # Define the prompt to ask the model
    model = "gemma:2b"
    prompt = f"""
    Given the following context:

    {context}

    Answer the following question:

    Question: {question}

    Provide a short and precise answer without any additional explanations.
    """

    print("Processing your question...")
    response = ollama.chat(model=model, messages=[{"role": "user", "content": prompt}])
    answer = response['message']['content']
    return answer.strip()

if __name__ == "__main__":
    check_cuda()

    paragraph = """
Hello everyone, welcome back to the computer networks course. Today we will see the classification of computer networks. After completing this session, we will understand what are LAN, MAN and WAN. And after understanding LAN, WAN and MAN, we will also understand what are the devices that are involved in LAN, MAN and WAN. And we will also understand what are the new trends in computer network.
    """

    # User input for the query
    user_query = input("Please enter your question related to the paragraph: ")

    # Get the answer from the model
    answer = ask_question(paragraph, user_query)

    # Display the result
    print("\nAnswer: ", answer)


CUDA is available. GPU in use: NVIDIA GeForce MX350
Processing your question...

Answer:  MAN refers to Metropolitan Area Network. It is a local area network that connects computers and devices within a city or town.
